# Drive characteristics using `strym`

In this notebook, we will generate characteristics of a particular drive. Please ensure strym is installed correctly

## Installing `strym`
First, we are required to install `strym`. For that, we will just use github devtools functionality.
Even if you have installed `strym` previously, you should reinstall for latest version of the package.

In [14]:
# Installing strym using pip from git
# !pip install gmplot gmaps googlemaps python-dotenv selenium m2r mkdocs pymdown-extensions sphinx-bootstrap-theme sphinx-markdown-parser webdriver-manager ytsphinx
#!pip install --upgrade --no-deps --force-reinstall git+https://github.com/jmscslgroup/strym
!pip install --upgrade --no-deps --force-reinstall strym==0.1.6

  Using cached https://files.pythonhosted.org/packages/4d/54/74615db9570531562ba02cc346a40f59dd367200b5cc3dd581a34ce14508/strym-0.1.6-py3-none-any.whl
  Found existing installation: strym 0.1.6
    Uninstalling strym-0.1.6:
      Successfully uninstalled strym-0.1.6


## Importing packages
Import required packages

In [1]:
from strym import strymread
import strym
import math
import time
import matplotlib.pyplot as plt
import numpy as np
import scipy.integrate as integrate

/Users/sprinkle/opt/anaconda3/lib/python3.7/site-packages/statsmodels/tools/_testing.py:19: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm


Loading BokehJS ...

## Specify Data Location

In [2]:
# datafolder = "/Users/sprinkle/work/data/cyverse/rahulbhadani/JmscslgroupData/PandaData/2020_03_03/"
# import glob
# csvlist = glob.glob(datafolder+"*.csv")
# datafolder = "/Users/sprinkle/work/data/cyverse/rahulbhadani/JmscslgroupData/PandaData/2020-03-05/"
# # print(csvlist)
# for f in glob.glob(datafolder+"*.csv"):
#     csvlist.append(f)
# print(csvlist)

In [4]:
import glob
parentfolder = "/Users/sprinkle/work/data/cyverse/rahulbhadani/JmscslgroupData/PandaData/"
folderlist = glob.glob(parentfolder+"*")
print(folderlist)
csvlist = []
for datafolder in folderlist:
#     datafolder = "/Users/sprinkle/work/data/cyverse/rahulbhadani/JmscslgroupData/PandaData/2020_03_03/"
    csvlisttmp = glob.glob(datafolder+"/*CAN*.csv")
    for f in csvlisttmp:
        csvlist.append(f)
#print(csvlist)

['/Users/sprinkle/work/data/cyverse/rahulbhadani/JmscslgroupData/PandaData/2020-03-11-10-18-08_CAN_Messages.csv', '/Users/sprinkle/work/data/cyverse/rahulbhadani/JmscslgroupData/PandaData/2019_10_25', '/Users/sprinkle/work/data/cyverse/rahulbhadani/JmscslgroupData/PandaData/2020_02_03', '/Users/sprinkle/work/data/cyverse/rahulbhadani/JmscslgroupData/PandaData/2020-03-10-10-57-06_GPS_Messages.csv', '/Users/sprinkle/work/data/cyverse/rahulbhadani/JmscslgroupData/PandaData/2020-03-10-11-04-13_GPS_Messages.csv', '/Users/sprinkle/work/data/cyverse/rahulbhadani/JmscslgroupData/PandaData/2020-03-11-16-43-24_GPS_Messages.csv', '/Users/sprinkle/work/data/cyverse/rahulbhadani/JmscslgroupData/PandaData/2020_03_03', '/Users/sprinkle/work/data/cyverse/rahulbhadani/JmscslgroupData/PandaData/2020_04_14', '/Users/sprinkle/work/data/cyverse/rahulbhadani/JmscslgroupData/PandaData/2020_03_04', '/Users/sprinkle/work/data/cyverse/rahulbhadani/JmscslgroupData/PandaData/2020-03-10-08-14-04_GPS_Messages.csv',

In [5]:
num_of_files = len(csvlist)
print("Total number of datafiles in {} is {}.".format(parentfolder, num_of_files))

Total number of datafiles in /Users/sprinkle/work/data/cyverse/rahulbhadani/JmscslgroupData/PandaData/ is 245.


In [6]:
# # testing time crap from python
# time_test = 1584107589.24185
# time2 = time.gmtime(time_test)
# print(time.strftime('%a %b %d %H:%M:%S %Y %Z',time2))

## Analysis
### 1. CSV file containing all messages
In this section, we will extract a subset of the CSV-formatted CAN Data in order to produce summary information of a specific drive.

In [7]:
import sys
import os
dbcfile = '../examples/newToyotacode.dbc'
metadata = []
for i in range(num_of_files):
    try:
        print(f'Processing {csvlist[i]}')
        val = 1
        if val==0:
            r0 = strymread(csvfile=csvlist[i], dbcfile=dbcfile)
            drive = r0.driving_characteristics()
        else:
            r0 = strymread(csvfile=csvlist[i], dbcfile=dbcfile)
            duration1 = r0.dataframe['Time'][len(r0.dataframe)-1] - r0.dataframe['Time'][0]
            duration_str = f'  Duration of this drive is {duration1} seconds ({math.trunc(int(duration1*1000)/(1000*60))} minutes {math.trunc(duration1 % 60)} seconds).'
            start_str = f'  Starting date/time of the drive is {time.ctime(r0.dataframe["Time"][0])}'

            # get the speed timeseries information from the data file
            speed_ts = r0.speed()
            # turn the timeseries into a python array for integration
            # transform from km/hr by multiplying 1000m/1km and 1 hr/3600s to get m/s
            speed_ar = np.array(speed_ts['Message'])*1000/3600
        #     speed_ar[0:-1]*1000/3600
            # find the difference of the time values
        #     dt = np.diff(np.array(speed_ts['Time']))
            dt = np.diff(speed_ts['Time'])
            # trapezoidal integration, divide by 1000 to get total km (rather than m)
            # Commented out: this produces negative values that don't make sense
        #     km_ts = np.trapz(y=speed_ar[0:-1],x=dt)/1000
            km_ts = np.trapz(y=speed_ar,x=np.array(speed_ts['Time']))/1000
            # commented out: this produces incorrect values that seem to be off by a factor of around 2, depending
            km_dx = np.trapz(y=speed_ar,dx=0.02)/1000
            # need to convert km/hr to km/s to get km later when integrating
            distance_str = f'  The trip was {km_ts} km ({km_ts*3.1/5} miles)'
        #     distance_str_dx f'  The dx version is {km_dx} km ({km_dx*3.1/5} miles)'

            #speed_ts['Time']
            #f'Total miles driven is {km_ts}'
        #     print(duration_str)
        #     print(start_str)
        #     print(distance_str)
        #     print('')
            # reproduce something like Fri Mar 13 06:53:09 2020 GMT-07
            time_str = (time.strftime('%a %b %d %H:%M:%S %Y %Z',time.gmtime(r0.dataframe['Time'][0])))
            # put it all together now
            drive = { 'filepath': csvlist[i], 'filename': os.path.basename(csvlist[i]), 'distance_km': km_ts, 'distance_miles': km_ts*3.1/5, 'duration_s': duration1, 
                     'date': time_str }
        metadata.append( drive )
    except:
        print(f'Unable to process ', csvlist[i], ', exception=', sys.exc_info() )
print(metadata)

Processing /Users/sprinkle/work/data/cyverse/rahulbhadani/JmscslgroupData/PandaData/2019_10_25/2019-10-25-13-51-36-833648_a4f27f0c-36b0-4abe-beed-3d548605c439_CAN_Message_Rav4.csv
Processing /Users/sprinkle/work/data/cyverse/rahulbhadani/JmscslgroupData/PandaData/2019_10_25/2019-10-25-13-57-22-031456_b118f877-5385-4205-894a-a37515307ee1_CAN_Message_Rav4.csv
Processing /Users/sprinkle/work/data/cyverse/rahulbhadani/JmscslgroupData/PandaData/2019_10_25/2019-10-25-13-49-54-310111_cb40aa45-de04-4313-bb9b-1bcf10ebaf70_CAN_Message_Rav4.csv
Processing /Users/sprinkle/work/data/cyverse/rahulbhadani/JmscslgroupData/PandaData/2019_10_25/2019-10-25-12-46-30-126677_8f806f3c-159e-4096-8552-b0aa7a0a681d_CAN_Message_Rav4.csv
Processing /Users/sprinkle/work/data/cyverse/rahulbhadani/JmscslgroupData/PandaData/2019_10_25/2019-10-25-13-57-42-883600_47016aab-73b8-470f-8c10-3e14aa85b582_CAN_Message_Rav4.csv
Processing /Users/sprinkle/work/data/cyverse/rahulbhadani/JmscslgroupData/PandaData/2019_10_25/2019-

/Users/sprinkle/opt/anaconda3/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3249: DtypeWarning: Columns (0,1,2,4) have mixed types.Specify dtype option on import or set low_memory=False.
  if (await self.run_code(code, result,  async_=asy)):


Unable to process  /Users/sprinkle/work/data/cyverse/rahulbhadani/JmscslgroupData/PandaData/0000_00_00/CAN_Data_Giraffe_2.csv , exception= (<class 'ValueError'>, ValueError("invalid literal for int() with base 10: 'MessageID'"), <traceback object at 0x125669e60>)
Processing /Users/sprinkle/work/data/cyverse/rahulbhadani/JmscslgroupData/PandaData/0000_00_00/CAN_Data_Giraffe_3.csv
Processing /Users/sprinkle/work/data/cyverse/rahulbhadani/JmscslgroupData/PandaData/0000_00_00/2018-01-28-09-00-07-789629__CAN_Message.csv
Processing /Users/sprinkle/work/data/cyverse/rahulbhadani/JmscslgroupData/PandaData/0000_00_00/CAN_Data_2.csv
Processing /Users/sprinkle/work/data/cyverse/rahulbhadani/JmscslgroupData/PandaData/0000_00_00/CAN_Data_Giraffe.csv
Processing /Users/sprinkle/work/data/cyverse/rahulbhadani/JmscslgroupData/PandaData/0000_00_00/CAN_Data.csv


/Users/sprinkle/opt/anaconda3/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3249: DtypeWarning: Columns (0,1,2,4) have mixed types.Specify dtype option on import or set low_memory=False.
  if (await self.run_code(code, result,  async_=asy)):


Unable to process  /Users/sprinkle/work/data/cyverse/rahulbhadani/JmscslgroupData/PandaData/0000_00_00/CAN_Data.csv , exception= (<class 'ValueError'>, ValueError("invalid literal for int() with base 10: 'MessageID'"), <traceback object at 0x1275d4190>)
Processing /Users/sprinkle/work/data/cyverse/rahulbhadani/JmscslgroupData/PandaData/0000_00_00/2018-01-28-09-00-52-647461__CAN_Message.csv
Processing /Users/sprinkle/work/data/cyverse/rahulbhadani/JmscslgroupData/PandaData/0000_00_00/2018-01-28-09-02-55-225012__CAN_Message.csv
Processing /Users/sprinkle/work/data/cyverse/rahulbhadani/JmscslgroupData/PandaData/0000_00_00/2018-01-28-09-07-22-781083__CAN_Message.csv
Processing /Users/sprinkle/work/data/cyverse/rahulbhadani/JmscslgroupData/PandaData/2020_03_24/2020-03-24-16-42-54_CAN_Messages.csv
Processing /Users/sprinkle/work/data/cyverse/rahulbhadani/JmscslgroupData/PandaData/2020_03_24/2020-03-24-09-03-10_CAN_Messages.csv
Processing /Users/sprinkle/work/data/cyverse/rahulbhadani/Jmscslg

In [8]:
# how did we do?
dist=0
for d in metadata:
    dist = dist + d['distance_miles']
print(dist)

912.4892436036764


In [9]:
metadata

[{'filepath': '/Users/sprinkle/work/data/cyverse/rahulbhadani/JmscslgroupData/PandaData/2019_10_25/2019-10-25-13-51-36-833648_a4f27f0c-36b0-4abe-beed-3d548605c439_CAN_Message_Rav4.csv',
  'filename': '2019-10-25-13-51-36-833648_a4f27f0c-36b0-4abe-beed-3d548605c439_CAN_Message_Rav4.csv',
  'distance_km': 0.0,
  'distance_miles': 0.0,
  'duration_s': 1.810054063796997,
  'date': 'Fri Oct 25 20:51:36 2019 UTC'},
 {'filepath': '/Users/sprinkle/work/data/cyverse/rahulbhadani/JmscslgroupData/PandaData/2019_10_25/2019-10-25-13-57-22-031456_b118f877-5385-4205-894a-a37515307ee1_CAN_Message_Rav4.csv',
  'filename': '2019-10-25-13-57-22-031456_b118f877-5385-4205-894a-a37515307ee1_CAN_Message_Rav4.csv',
  'distance_km': 0.0,
  'distance_miles': 0.0,
  'duration_s': 0.9986419677734375,
  'date': 'Fri Oct 25 20:57:22 2019 UTC'},
 {'filepath': '/Users/sprinkle/work/data/cyverse/rahulbhadani/JmscslgroupData/PandaData/2019_10_25/2019-10-25-13-49-54-310111_cb40aa45-de04-4313-bb9b-1bcf10ebaf70_CAN_Messag

In [10]:
import json
import os
import datetime
# eventually do everything, but for now just one file
for drive_i in metadata:
    # drive_i = metadata[215]
    drive_i["metadata_date"] = datetime.datetime.now().astimezone().strftime('%a %b %d %H:%M:%S %Y %Z')
    # find the name attached to this filename
    file_path = os.path.splitext(drive_i["filepath"])[0]
    index = file_path.find("CAN")
    json_filename = f'{file_path[0:index]}_metadata.json'

    #json_filename = f'{}.json'
    print(json_filename)
    ## json_basename = os.path.basename(drive_i['filename'])
    with open(json_filename,'w') as outfile:
        json.dump(drive_i,outfile,indent=4)

/Users/sprinkle/work/data/cyverse/rahulbhadani/JmscslgroupData/PandaData/2019_10_25/2019-10-25-13-51-36-833648_a4f27f0c-36b0-4abe-beed-3d548605c439__metadata.json
/Users/sprinkle/work/data/cyverse/rahulbhadani/JmscslgroupData/PandaData/2019_10_25/2019-10-25-13-57-22-031456_b118f877-5385-4205-894a-a37515307ee1__metadata.json
/Users/sprinkle/work/data/cyverse/rahulbhadani/JmscslgroupData/PandaData/2019_10_25/2019-10-25-13-49-54-310111_cb40aa45-de04-4313-bb9b-1bcf10ebaf70__metadata.json
/Users/sprinkle/work/data/cyverse/rahulbhadani/JmscslgroupData/PandaData/2019_10_25/2019-10-25-12-46-30-126677_8f806f3c-159e-4096-8552-b0aa7a0a681d__metadata.json
/Users/sprinkle/work/data/cyverse/rahulbhadani/JmscslgroupData/PandaData/2019_10_25/2019-10-25-13-57-42-883600_47016aab-73b8-470f-8c10-3e14aa85b582__metadata.json
/Users/sprinkle/work/data/cyverse/rahulbhadani/JmscslgroupData/PandaData/2019_10_25/2019-10-25-13-39-54-897963_4acda047-6dee-4398-907b-3a40c3468e05__metadata.json
/Users/sprinkle/work/d